## Optical Rain gauges and Tipping Bucket Rain Gauge Processing

This script will process the ORG and APG data generatored at ATMOS. Additionally data from the Nova Lynx 12 inch rain gauge and Belfort 8in rain may also be present in specific data ranges. 

# Importing libraries

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import act

D:\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Defining metadata

The data output from the ORG and APG has no meta data and doesn't contain units. We will be adding some metadata that are consistent through all files. The rain gauge metadata will need to be added and adjusted based on the known file. 

In [3]:
attrs_dict = {'ORG_PW_Code': {'standard_name': 'WMO Present Weather Condition Code'},
              'ORG_Precip_Rate': {'standard_name': 'Instantaneous Precipitation Rate',
                               'units':'mm/hr'},
              'ORG_Precip_Accum': {'standard_name': 'Precipitation Accumulation',
                          'units': 'mm'},
              'ORG_Temp': {'standard_name': 'ORG Temperature Probe',
                           'units':'degC'},
              'APG_PW_Code': {'standard_name': 'WMO Present Weather Condition Code'},
              'APG_Precip_Rate': {'standard_name': 'Instantaneous Precipitation Rate',
                                  'units':'mm/hr'},
              'APG_Precip_Accum': {'standard_name': 'Precipitation Accumulation',
                                   'units':'mm'},
              'APG_Temp': {'standard_name': 'ORG Temperature Probe',
                        'units': 'degC'}}


# Reading the file, re-indexing and reassigning type 

This filepath is specific to the computer I've ran the script on and will need to be modified for your use. Skiprows in the reader skips extra information that is not useful to when reading in the data.

In [4]:
df = pd.read_csv('N:/home/work/ORG_APG/20230606_ORG_APG.dat',
                 skiprows=[0, 2, 3])

Now changing the time variable to a datetime64 type

In [5]:
df['time'] = df['TIMESTAMP'].astype('datetime64')

Now we'll change how the file is indexed to be off of the time variable rather than a line number. We'll also delete the TIMESTAMP column, as it becomes redundent. 

In [6]:
df.set_index('time', inplace=True)
del df['TIMESTAMP']

The data at this point reads in as an object rather than a float. The next section changes the specific columns to the defined datatype we want. 

In [7]:
df[['RECORD', 'ORG_Precip_Rate', 'ORG_Precip_Accum', 'ORG_Stat',
    'ORG_Car', 'ORG_Raw', 'ORG_Base', 'ORG_Temp', 'APG_Precip_Rate', 
    'APG_Precip_Accum', 'APG_Stat', 'APG_Car', 'APG_Raw',
    'APG_Base', 'APG_Temp']] = df[['RECORD', 'ORG_Precip_Rate',
                                   'ORG_Precip_Accum', 'ORG_Stat',
                                   'ORG_Car', 'ORG_Raw', 'ORG_Base', 'ORG_Temp',
                                   'APG_Precip_Rate', 
                                   'APG_Precip_Accum', 'APG_Stat', 'APG_Car', 
                                   'APG_Raw','APG_Base', 
                                   'APG_Temp']].apply(pd.to_numeric, errors='coerce')

# Converting to Xarray Dataset and Adding Attributes

To make it easier to work with, we'll convert to an xarray dataset

In [8]:
ds = df.to_xarray()

In [9]:
ds

<xarray.Dataset>
Dimensions:              (time: 30177)
Coordinates:
  * time                 (time) datetime64[ns] 2023-05-16T19:55:00 ... 2023-0...
Data variables: (12/19)
    RECORD               (time) int64 0 1 2 3 4 ... 30173 30174 30175 30176
    ORG_PW_Code          (time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    ORG_Precip_Rate      (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_Precip_Accum     (time) float64 0.0 0.0 0.0 0.0 ... 0.007 0.007 0.007
    ORG_Stat             (time) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    ORG_Car              (time) int64 4999 4999 4999 4999 ... 4999 4999 4999
    ...                   ...
    APG_Car              (time) float64 4.999e+03 4.999e+03 ... 4.999e+03
    APG_Raw              (time) float64 40.0 40.0 40.0 40.0 ... 71.0 64.0 67.0
    APG_Base             (time) float64 141.0 137.0 134.0 ... 116.0 114.0 112.0
    APG_Temp             (time) float64 27.0 27.0 27.0 27.0 ... 22.0 22.0 22.0
    tb_rain_mm_12in_Tot  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tb_rain_mm_8in_Tot   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

Now that we've printed out our dataset, we see that the variables exist but have no units. Time to loop through the data to add the attributes defined earlier

In [14]:
for variable in attrs_dict.keys():
    if variable in list(ds.variables):
        ds[variable].attrs = attrs_dict[variable]


# Decoding Present Weather Codes

The APG and ORG both output present weather codes that can give the user an idea of what type of precipitation was falling at the given time. While these codes can be useful, the instruments are limited to 10 present weather codes and may not be accurate. 

To decode these present weather codes, we can use ACT, which already has a built in function to handle the WMO table 4680 weather codes. In the ACT utility, you don't need to define a variable for it be renamed into, it automatically appends the xarray dataset with a new coded variable. 

First we'll do the ORG present weather code

In [15]:
act.utils.decode_present_weather(ds, variable='ORG_PW_Code')

<xarray.Dataset>
Dimensions:              (time: 30177)
Coordinates:
  * time                 (time) datetime64[ns] 2023-05-16T19:55:00 ... 2023-0...
Data variables: (12/21)
    RECORD               (time) int64 0 1 2 3 4 ... 30173 30174 30175 30176
    ORG_PW_Code          (time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    ORG_Precip_Rate      (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_Precip_Accum     (time) float64 0.0 0.0 0.0 0.0 ... 0.007 0.007 0.007
    ORG_Stat             (time) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    ORG_Car              (time) int64 4999 4999 4999 4999 ... 4999 4999 4999
    ...                   ...
    APG_Base             (time) float64 141.0 137.0 134.0 ... 116.0 114.0 112.0
    APG_Temp             (time) float64 27.0 27.0 27.0 27.0 ... 22.0 22.0 22.0
    tb_rain_mm_12in_Tot  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tb_rain_mm_8in_Tot   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_PW_Code_decoded  (time) <U31 'No significant weather observed' ... 'N...
    APG_PW_Code_decoded  (time) <U37 'No significant weather observed' ... 'N...

In [16]:
act.utils.decode_present_weather(ds, variable='APG_PW_Code')

<xarray.Dataset>
Dimensions:              (time: 30177)
Coordinates:
  * time                 (time) datetime64[ns] 2023-05-16T19:55:00 ... 2023-0...
Data variables: (12/21)
    RECORD               (time) int64 0 1 2 3 4 ... 30173 30174 30175 30176
    ORG_PW_Code          (time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    ORG_Precip_Rate      (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_Precip_Accum     (time) float64 0.0 0.0 0.0 0.0 ... 0.007 0.007 0.007
    ORG_Stat             (time) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    ORG_Car              (time) int64 4999 4999 4999 4999 ... 4999 4999 4999
    ...                   ...
    APG_Base             (time) float64 141.0 137.0 134.0 ... 116.0 114.0 112.0
    APG_Temp             (time) float64 27.0 27.0 27.0 27.0 ... 22.0 22.0 22.0
    tb_rain_mm_12in_Tot  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tb_rain_mm_8in_Tot   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_PW_Code_decoded  (time) <U31 'No significant weather observed' ... 'N...
    APG_PW_Code_decoded  (time) <U37 'No significant weather observed' ... 'N...

# Tipping Bucket Data

The above section is standard for all files since the data set started in March. However, 2 tipping bucket rain gauges have been added at different times. Both began on May 16th. The section below will process that data for files after May 16th.  

In [23]:
ds.tb_rain_mm_12in_Tot.attrs['standard_name']='12 inch Tipping Bucket Rain Gauge'
ds.tb_rain_mm_12in_Tot.attrs['units'] = 'mm'

ds.tb_rain_mm_8in_Tot.attrs['standard_name']='8 inch Tipping Bucket Rain Gauge'
ds.tb_rain_mm_8in_Tot.attrs['units'] = 'mm'

In [24]:
ds


<xarray.Dataset>
Dimensions:              (time: 30177)
Coordinates:
  * time                 (time) datetime64[ns] 2023-05-16T19:55:00 ... 2023-0...
Data variables: (12/21)
    RECORD               (time) int64 0 1 2 3 4 ... 30173 30174 30175 30176
    ORG_PW_Code          (time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    ORG_Precip_Rate      (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_Precip_Accum     (time) float64 0.0 0.0 0.0 0.0 ... 0.007 0.007 0.007
    ORG_Stat             (time) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    ORG_Car              (time) int64 4999 4999 4999 4999 ... 4999 4999 4999
    ...                   ...
    APG_Base             (time) float64 141.0 137.0 134.0 ... 116.0 114.0 112.0
    APG_Temp             (time) float64 27.0 27.0 27.0 27.0 ... 22.0 22.0 22.0
    tb_rain_mm_12in_Tot  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tb_rain_mm_8in_Tot   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_PW_Code_decoded  (time) <U31 'No significant weather observed' ... 'N...
    APG_PW_Code_decoded  (time) <U37 'No significant weather observed' ... 'N...

# Outputting File Per Day

Above will work with as many days of data as given in each file. We will want to break the data up by day to limit how many data we need to load in at times. To do this, we'll loop through the data, extract out unique days. Additionally, it'll also create a cumulative sum of the tbrg rain fall to add in each file and create a daily rainfall for the optical gauges, which otherwise plot a running total.

A checker needs to be added to remove any negative precip values, as the ORG tends to reset to zero if it loses power.

In [25]:
for i in np.unique(ds.time.dt.strftime('%Y%m%d')):
    daily_data = ds.sel(time=i)  
    daily_data = daily_data.assign(tb_12in_precip_mm_accum=
                                   daily_data.tb_rain_mm_12in_Tot.cumsum())
    daily_data['tb_12in_precip_mm_accum'].attrs['units'] = 'mm'
    
    daily_data = daily_data.assign(tb_8in_precip_mm_accum=
                                   daily_data.tb_rain_mm_8in_Tot.cumsum())
    daily_data['tb_8in_precip_mm_accum'].attrs['units'] = 'mm'

    daily_data = daily_data.assign(
        corr_precip_ORG=daily_data.ORG_Precip_Accum-daily_data.ORG_Precip_Accum[0])
    daily_data['corr_precip_ORG'].attrs['units'] = 'mm'

    daily_data = daily_data.assign(
       corr_precip_APG=daily_data.APG_Precip_Accum-daily_data.APG_Precip_Accum[0])
    daily_data['corr_precip_APG'].attrs['units'] = 'mm'
    daily_data.to_netcdf(i+'_ORG_APG.nc')
    print(i)


20230516
20230517
20230518
20230519
20230520
20230521
20230522
20230523
20230524
20230525
20230526
20230527
20230528
20230529
20230530
20230531
20230601
20230602
20230603
20230604
20230605
20230606


In [26]:
daily_data

<xarray.Dataset>
Dimensions:                  (time: 1132)
Coordinates:
  * time                     (time) datetime64[ns] 2023-06-06 ... 2023-06-06T...
Data variables: (12/25)
    RECORD                   (time) int64 29045 29046 29047 ... 30175 30176
    ORG_PW_Code              (time) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    ORG_Precip_Rate          (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ORG_Precip_Accum         (time) float64 4.003 4.003 4.003 ... 0.007 0.007
    ORG_Stat                 (time) int64 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ORG_Car                  (time) int64 4999 4999 4999 4999 ... 4999 4999 4999
    ...                       ...
    ORG_PW_Code_decoded      (time) <U31 'No significant weather observed' .....
    APG_PW_Code_decoded      (time) <U37 'No significant weather observed' .....
    tb_12in_precip_mm_accum  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tb_8in_precip_mm_accum   (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    corr_precip_ORG          (time) float64 0.0 0.0 0.0 ... -3.996 -3.996 -3.996
    corr_precip_APG          (time) float64 0.0 0.0 0.0 ... -0.001 -0.001 -0.001